In [3]:
from tensorflow import keras

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("./model/unet_model.h5")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

def predict_contour_save(image_path, model, output_text_file="../output/coordinates.txt"):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize image to match model input shape
    input_size = (128, 128)  # Adjust according to your model's input size
    resized_image = cv2.resize(image, input_size) / 255.0  # Normalize
    
    # Predict mask
    pred_mask = model.predict(np.expand_dims(resized_image, axis=0))[0]
    pred_mask = (pred_mask > 0.5).astype(np.uint8) * 255  # Binarize

    # Find contours
    contours, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw contours on original image
    contour_image = image.copy()
    cv2.drawContours(contour_image, contours, -1, (255, 0, 0), 2)  # Green contours

    # Save contour coordinates to text file
    coordinates = []
    for contour in contours:
        contour_points = contour.squeeze(axis=1)  # Removing extra dimension (n,1,2) to (n,2)
        coordinates.append(contour_points.tolist())

    # Write coordinates to a file
    with open(output_text_file, "w") as file:
        for contour in coordinates:
            #file.write("Contour:\n")
            for point in contour:
                file.write(f"{point[0]},{point[1]}\n")
            file.write("\n")

    print(f"Contour coordinates saved to {output_text_file}")


    # Show results
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")

    plt.subplot(1, 3, 2)
    plt.imshow(pred_mask, cmap="gray")
    plt.title("Predicted Mask")

    plt.subplot(1, 3, 3)
    plt.imshow(contour_image)
    plt.title("Contour Overlay")

    plt.show()

In [ ]:
# Test on a new image
predict_contour_save("../dataset_portion/new_image.jpg", model)

In [ ]:
# Load the image
image = cv2.imread("../dataset_portion/new_image.jpg")

# Read the coordinates from the file
contours = []
with open("../output/coordinates.txt", "r") as f:
    contour = []
    for line in f:
        line = line.strip()
        if line:
            x, y = map(int, line.split(","))
            contour.append([x, y])
        else:
            if contour:  # If a contour exists, add it and reset
                contours.append(np.array(contour, dtype=np.int32))
                contour = []

# Draw contours on the image
cv2.drawContours(image, contours, -1, (255, 0, 0), 1)  # Blue color contours

# Save or display the result
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#cv2.imwrite("output/image_with_contours.jpg", image)
plt.title("Loaded Pool Contours")
plt.show()